In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import pydicom
import cv2
import os

# Function to load, preprocess images, and assign labels
def load_and_preprocess_images_with_labels(directory):
    image_data = []
    labels = []

    image_size_fmri = (75, 75)  
    image_size_mri = (128, 128) 
    image_size_pet = (128, 128) 

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size_fmri)  # Adjust image size as needed
                image_data.append(img)
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

data_dir = r'D:\Alzheimers\Voting2'

test_data, true_labels = load_and_preprocess_images_with_labels(data_dir)


model_fmri = load_model('D:/Alzheimers/Models/fMRI_Task1.h5')
model_mri = load_model('D:/Alzheimers/Models/MRI_Task1.h5')
model_pet = load_model('D:/Alzheimers/Models/PET_Task1.h5')

test_data = np.repeat(test_data[:, :, :, np.newaxis], 3, axis=-1) 

test_data_fmri_resized = []
for image in test_data:
    resized_image = cv2.resize(image, (75, 75))  
    test_data_fmri_resized.append(resized_image)
test_data_fmri_resized = np.array(test_data_fmri_resized)
print(test_data_fmri_resized.shape) 

predictions_fmri = model_fmri.predict(test_data_fmri_resized)

test_data_mri_resized = []
for image in test_data:
    resized_image = cv2.resize(image, (128, 128)) 
    test_data_mri_resized.append(resized_image)
test_data_mri_resized = np.array(test_data_mri_resized)
print(test_data_mri_resized.shape) 

predictions_mri = model_mri.predict(test_data_mri_resized)

test_data_pet_resized = [] 
for image in test_data:
    resized_image = cv2.resize(image, (128, 128))
    test_data_pet_resized.append(resized_image)
test_data_pet_resized = np.array(test_data_pet_resized)
print(test_data_pet_resized.shape) 

predictions_pet = model_pet.predict(test_data_pet_resized)

def majority_vote(predictions_fmri, predictions_mri, predictions_pet):
    predictions = []
    for i in range(len(predictions_fmri)):
        combined_prediction = np.argmax([predictions_fmri[i], predictions_mri[i], predictions_pet[i]])
        predictions.append(combined_prediction % 5)  
    return predictions

final_predictions = majority_vote(predictions_fmri, predictions_mri, predictions_pet)

accuracy = accuracy_score(true_labels, final_predictions)

predicted_labels_mri = np.argmax(predictions_mri, axis=1)
predicted_labels_fmri = np.argmax(predictions_fmri, axis=1)
predicted_labels_pet = np.argmax(predictions_pet, axis=1)

print(f"Voting Classifier Accuracy: {accuracy * 100:.2f}%")

(50000, 75, 75, 3)
1563/1563 [==============================] - 407s 260ms/step
(50000, 128, 128, 3)
1563/1563 [==============================] - 737s 468ms/step
(50000, 128, 128, 3)
1563/1563 [==============================] - 730s 465ms/step
Voting Classifier Accuracy: 60.87%
